In [ ]:
#Sem transfer learning

import os
import cv2
import glob as gb
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix


In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
SEED = 1000
IMG_SIZE = 256
BATCH_SIZE = 64

TRAIN_DIR = '/content/drive/MyDrive/Test'
class_names = []
class_count = []
train_examples = 0
for f in os.listdir(TRAIN_DIR):
    files = gb.glob(pathname=str(TRAIN_DIR + "//" + f + "/*"))
    class_names.append(f)
    class_count.append(len(files))
    train_examples += len(files)
plt.figure(figsize=(20, 5))
sns.barplot(x=class_names, y=class_count)
plt.savefig('foo.png', dpi=1000)
plt.show()

In [ ]:
train_gen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3
)

train_batch = train_gen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical',
    seed=SEED
)

valid_batch = train_gen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode='categorical',
    seed=SEED
)

base_model = tf.keras.applications.DenseNet121(include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(200, activation='softmax')
])

model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    metrics=['accuracy']
)

cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.02, patience=10, restore_best_weights=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001, verbose=2)

In [ ]:
history = model.fit(
    train_batch,
    epochs=20,
    steps_per_epoch=train_examples // BATCH_SIZE,
    validation_data=valid_batch,
    validation_steps=len(valid_batch),
    callbacks=[cb, reduce_lr]
)

In [ ]:
model.save("/content/drive/MyDrive/densenet_model.h5")

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.ylim([0, 10])
plt.legend(loc='upper right')
plt.savefig("densenet.pdf", dpi=300)
plt.show()

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/densenet_model.h5')

In [ ]:
y_pred = model.predict(valid_batch)
y_pred = np.argmax(y_pred, axis=1)

y_true = valid_batch.classes

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig("confusion_matrix.png", dpi=300)
plt.show()

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image

In [ ]:
image_path = '/content/drive/MyDrive/Test/American_Pipit/American_Pipit_0001_100225.jpg'
preprocessed_image = preprocess_image(image_path)
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

predictions = model.predict(preprocessed_image)
class_indices = train_batch.class_indices
class_mapping = {v: k for k, v in class_indices.items()}
predicted_class_index = np.argmax(predictions)
predicted_class_name = class_mapping[predicted_class_index]
print("Predicted class:", predicted_class_name)